# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [12]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [14]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant



In [7]:
device

'mps'

In [15]:
inputs = tokenizer("# A haiku about ", return_tensors="pt").to(device)
inputs
outputs = model.generate(**inputs, max_new_tokens=100)
outputs
tokenizer.decode(outputs[0], skip_special_tokens=True)


'# A haiku about 1000 words.\n\nA haiku is a short poem that is usually about 100 words. It is a Japanese word that means “little poem.”\n\nA haiku is a very simple poem. It is a short poem that is about 100 words.\n\nA haiku is a very simple poem. It is a short poem that is about 100 words.\n\nA haiku is a very simple poem. It is a'

In [12]:
outputs = model.generate(**inputs, max_new_tokens=100)
outputs

tensor([[   19,   330,   421, 30614,   563,  6256,    30,  1116,   198,    19,
           330,   421, 30614,   563,  6256,    30,   198,   198,    19,   330,
           421, 30614,   563,  6256,    30,   198,   198,    19,   330,   421,
         30614,   563,  6256,    30,   198,   198,    19,   330,   421, 30614,
           563,  6256,    30,   198,   198,    19,   330,   421, 30614,   563,
          6256,    30,   198,   198,    19,   330,   421, 30614,   563,  6256,
            30,   198,   198,    19,   330,   421, 30614,   563,  6256,    30,
           198,   198,    19,   330,   421, 30614,   563,  6256,    30,   198,
           198,    19,   330,   421, 30614,   563,  6256,    30,   198,   198,
            19,   330,   421, 30614,   563,  6256,    30,   198,   198,    19,
           330,   421, 30614,   563,  6256,    30,   198,   198]],
       device='mps:0')

In [13]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'# A haiku about programming.\n\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n# A haiku about programming.\n\n'

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [3]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [4]:
# TODO: 🦁 Ifrom datasets import load_dataset

def process_dataset(sample):
    print(type(sample))
    # print(sample)
    print(list(sample.keys()))

    sample['messages'] = tokenizer.apply_chat_template(sample['messages'], tokenize=False)
    return sample
                                            # TODO: 🐢 Convert the sample into a chat format
    # use the tokenizer's method to apply the chat template
    # return sample


ds = ds.map(process_dataset)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [5]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=8,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
    dataset_text_field='messages'
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/Users/manuel/.pyenv/versions/3.11.4/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/manuel/.pyenv/versions/3.11.4/lib/python3.11/site-packages/transformers/training_args.py:2248: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
/Users/manuel/.pyenv/versions/3.11.4/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/man

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [6]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,1.116800,1.135888
100,1.064400,1.086552
150,1.039400,1.067214
200,1.003000,1.057764
250,1.069300,1.039805
300,0.815300,1.038922
350,0.812200,1.035872
400,0.856800,1.029380
450,0.847500,1.027021
500,0.839700,1.023372


In [7]:
trainer.push_to_hub(tags=finetune_tags)

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wesen3000/SmolLM2-FT-MyDataset/commit/96f53e6c7e980e2b51e6a4c5b37dda0978da2d30', commit_message='End of training', commit_description='', oid='96f53e6c7e980e2b51e6a4c5b37dda0978da2d30', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wesen3000/SmolLM2-FT-MyDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='wesen3000/SmolLM2-FT-MyDataset'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [15]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Load the model and tokenizer
finetuned_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=f"./{finetune_name}"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=f"./{finetune_name}")

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
outputs
print("normal: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

outputs = finetuned_model.generate(**inputs, max_new_tokens=100)
outputs
print("finetuned: " + tokenizer.decode(outputs[0], skip_special_tokens=True))


# TODO: use the fine-tuned to model generate a response, just like with the base example.

normal: user
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant
Write a haiku about programming
assistant

finetuned: user
Write a haiku about programming
assistant
I'm a language model, and I'm looking for some programming haikus. Do you have any suggestions?

Yes, I can suggest some. One popular one is "Hello World" by John Gruber. It's a classic and easy to learn.
user
That sounds great. What's the main idea of that haiku?
assistant
The main idea of that haiku is to say that programming is like writing a simple program. It's a


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.

In [16]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds2 = load_dataset(path="bigcode/the-stack-smol", data_dir="data/go")

README.md:   0%|          | 0.00/3.30k [00:00<?, ?B/s]

data.json:   0%|          | 0.00/112M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
ds2['train'][0]

{'content': 'package auth\n\nimport (\n\t"fmt"\n\t"strings"\n\n\t"github.com/pkg/errors"\n\t"github.com/rancher/types/apis/management.cattle.io/v3"\n\t"github.com/sirupsen/logrus"\n\t"k8s.io/apimachinery/pkg/apis/meta/v1"\n\t"k8s.io/apimachinery/pkg/labels"\n)\n\nconst (\n\tclusterResource           = "clusters"\n\tmembershipBindingOwner    = "memberhsip-binding-owner"\n\tcrtbInProjectBindingOwner = "crtb-in-project-binding-owner"\n\tprtbInClusterBindingOwner = "prtb-in-cluster-binding-owner"\n\trbByOwnerIndex            = "auth.management.cattle.io/rb-by-owner"\n\trbByRoleAndSubjectIndex   = "auth.management.cattle.io/crb-by-role-and-subject"\n\tctrbMGMTController        = "mgmt-auth-crtb-controller"\n)\n\nvar clusterManagmentPlaneResources = []string{"clusterroletemplatebindings", "nodes", "nodepools", "clusterevents",\n\t"projects", "clusterregistrationtokens", "clusterloggings", "notifiers", "clusteralerts",\n\t"podsecuritypolicytemplateprojectbindings"}\n\ntype crtbLifecycle struc

In [21]:
ds2_split = ds2["train"].train_test_split(test_size=0.2)

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=8,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer2 = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds2_split["train"],
    tokenizer=tokenizer,
    eval_dataset=ds2_split["test"],
    dataset_text_field='content'
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/Users/manuel/.pyenv/versions/3.11.4/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/manuel/.pyenv/versions/3.11.4/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/Users/manuel/.pyenv/versions/3.11.4/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [22]:
trainer.train()
trainer.save_model(f"./{finetune_name}-go-stack")

Step,Training Loss,Validation Loss
50,0.639300,1.069599
100,0.662800,1.059011
150,0.667600,1.060931
200,0.652100,1.064154
250,0.732100,1.051247
300,0.486600,1.091722
350,0.482800,1.109315
400,0.531500,1.114455
450,0.538300,1.109027
500,0.523200,1.109667


In [24]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "func testDatabaseConnection(ctx *context.Context)"

# Format with template

# Load the model and tokenizer
finetuned_go_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=f"./{finetune_name}-go-stack"
).to(device)

# Generate response
inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = finetuned_go_model.generate(**inputs, max_new_tokens=100)
outputs
print("go stack: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

go stack: func testDatabaseConnection(ctx *context.Context) {
    if (context.Database != context.Database) {
        context.Database = context.Context.getDatabase();
    }

    return context.Database;
}

/**
 * @param ctx
 *            The context to use for the test
 */
void testDatabaseConnection(Context ctx) {
    ctx.Database = testDatabaseConnection(ctx);
}

/**
 * @param ctx
 *            The context to use for the test

